# Traing and deploying models with Azure Machine Learning
This tutorial covers the following
- Import data from blob url
- Train a Scikit learn linear regression model with MLflow and track it in AML
- Register the trained model using MLflow into AML Model registry
- Deploy registered model to an AML Managed Endpoint
- Perform Inference againg the deployed Model

In this example you will build a model to predict quality of wine based on quantitative features like the wine’s “fixed acidity”, “pH”, “residual sugar”, and so on. The dataset is from UCI’s machine learning repository.


# Requirements

- An Azure Machine Learning workspace. For more information, see [Create an Azure Machine Learning workspace](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-workspace).
- An Azure Machine Learning Compute Instance. For more information, see [Create an Compute Instance](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-manage-compute-instance?tabs=azure-studio#create) or use an existing Compute Instance.
- The Azure CLI extension for [Machine Learning Service (v2)](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-cli?tabs=public). It is pre-installed on Compute Instance
  - To ensure no version conflict: for "name: ml", version should be "2.x"
- Upload this notebook to Compute Instance by clicking on **Upload Files** instead of **Create new file** option as shown in [how to manage files in Compute Instance](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-files)
- Accessing terminal on Compute Instance. For more information, see [Accessing a compute instance terminal](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-access-terminal)


# Setup


In [ ]:
# enter details of your AML workspace
subscription_id = "b17253fa-f327-42d6-9686-f3e553e24763" #"Data Science VM Team"
resource_group = "mjn-dpv2-cli"
workspace = "mjn-dpv2"

model_name = "ElasticnetWineModel-7-11"
endpoint_name = "mjn-wine-quality-7-11"
deployment_name="blue-7-11"

# Train and Register the model


**Import from a blob storage via URL**

In [ ]:
import pandas as pd

# Read the wine-quality csv file from the URL
csv_url =\
    'https://akannava5717179148.blob.core.windows.net/publicdata/winequality-red.csv'
try:
    data = pd.read_csv(csv_url, sep=';')
except Exception as e:
    print("Unable to download training & test CSV, check your internet connection. Error: %s", e)

In [ ]:
data.head()

**Split data into two sets**
- Training (75% of dataset used to traing model)
- Validation (25% of dataset used for validation)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets. (0.75, 0.25) split.
 
X = data.drop(["quality"], axis=1)
y = data.quality
 
# Split out the training data
train_x, test_x, train_y, test_y = train_test_split(X, y, train_size=0.75)

**Train Scikit liner regression model with MLflow**

Train [sklearn.linear_model.ElasticNet](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) (uses mlflow autolog feature to track metrics). 

In [ ]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet

np.random.seed(40)

# Enable autolog to log metrics, params, artifacts, models automatically
mlflow.autolog(log_input_examples=True)

alpha = 0.5
l1_ratio = 0.5

with mlflow.start_run() as run:
    # Execute ElasticNet
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

print("Run id is {}".format(run.info.run_id))

**Register in Model Registry**

So that it can be used for inferencing.

In [ ]:
model_version  = mlflow.register_model(
        "runs:/{}/model".format(run.info.run_id),
        model_name
    )

## Inference

To productionize the model for predictions, use [Managed endpoints](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-managed-online-endpoints) is an AzureML product that manages the inferencing infrastructure for customers, so that they can focus on their business logic. 

An endpoint is like a server customers can call with input data to get a prediction. The endpoint is also like a traffic manager where there can be multiple deployments behind it. Each deployment has different configuration on which model to use, what's the environment and how to invoke the model to get the prediction result.


In [ ]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes

# get a handle to the workspace
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

**Create an Endpoint**

In [ ]:
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="this is a sample online endpoint for mlflow model",
    auth_mode="key",
    tags={"foo": "bar"},
)
ml_client.begin_create_or_update(endpoint)

**Create Deployment**

In [ ]:
# az ml online-deployment create -f mir-deployment.yml --all-traffic

model = ml_client.models.get(name=model_name, label="latest");
blue_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model=model,
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

ml_client.online_deployments.begin_create_or_update(blue_deployment)

# blue deployment takes 100 traffic
endpoint.traffic = {deployment_name: 100}
ml_client.begin_create_or_update(endpoint)

**Create request payload file**

In [ ]:
%%writefile request-file.json
{
    "input_data": {
        "columns": [
            "alcohol",
            "chlorides",
            "citric acid",
            "density",
            "fixed acidity",
            "free sulfur dioxide",
            "pH",
            "residual sugar",
            "sulphates",
            "total sulfur dioxide",
            "volatile acidity"
        ],
        "data": [[7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4]]
    }
}

**Invoke the enpoint**

It should return a reponse like **"\[5.570454783358535\]"** giving a number indicating quality of wine

In [ ]:
## az ml online-endpoint invoke --name <endpoint_name> --request-file request-file.json

ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name=deployment_name,
    request_file="request-file.json",
)


**Delete the deployment and endpoint**

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)